In [ ]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt


def label_images(start_index, end_index, image_folder, output_file):
    distortion_criteria = ["lighting", "focus", "orientation", "color_calibration", "background", "resolution", "field_of_view"]
    scores = {}
    if os.path.exists(output_file):
        with open(output_file, 'r') as file:
            scores = json.load(file)

    all_images = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    selected_images = all_images[start_index:end_index]

    for filename in selected_images:
        image_path = os.path.join(image_folder, filename)
        image = Image.open(image_path)
        
        plt.figure(figsize=(8, 6))
        plt.imshow(image)
        plt.axis('off')
        plt.show()

        image_scores = {}
        for criterion in distortion_criteria:
            valid_score = False
            while not valid_score:
                try:
                    score = float(input(f"Enter score (0-1) for {criterion}: "))
                    if 0 <= score <= 1:
                        image_scores[criterion] = score
                        valid_score = True
                    else:
                        print("Score must be between 0 and 1. Please try again.")
                except ValueError:
                    print("Invalid input. Please enter a numerical value.")
        
        scores[filename] = image_scores

    with open(output_file, "w") as json_file:
        json.dump(scores, json_file)

    print("Scores saved successfully.")

In [ ]:
label_images(190, 200, "to_label", "scores.json")

In [ ]:
import json

with open("scores.json", "r") as json_file:
    scores = json.load(json_file)
len(scores.keys())

In [ ]:
import json
import matplotlib.pyplot as plt

def plot_score_distribution(output_file):
    with open(output_file, 'r') as file:
        scores = json.load(file)

    distortion_criteria = ["lighting", "focus", "orientation", "color_calibration", "background", "resolution", "field_of_view"]
    criteria_scores = {criterion: [] for criterion in distortion_criteria}
    for image_scores in scores.values():
        for criterion, score in image_scores.items():
            criteria_scores[criterion].append(score)

    _, axs = plt.subplots(3, 3, figsize=(15, 10))
    axs = axs.flatten()

    for idx, criterion in enumerate(distortion_criteria):
        axs[idx].hist(criteria_scores[criterion], bins=10, range=(0, 1), color='#C68642', edgecolor='black')
        axs[idx].set_title(f'Distribution of {criterion} Scores')
        axs[idx].set_xlabel('Score')
        axs[idx].set_ylabel('Frequency')

    for ax in axs[len(distortion_criteria):]:
        ax.set_visible(False)

    plt.tight_layout()
    plt.show()

plot_score_distribution('scores.json')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plot_grouped_distributions_from_npy(file_path):
    scores = np.load(file_path)  # Load the .npy file

    distortion_criteria = ['Background', 'Lighting', 'Focus', 'Orientation', 'Color calibration', 'Resolution', 'Field of view']
    num_images, num_criteria = scores.shape

    assert num_criteria == len(distortion_criteria), "The number of criteria in the .npy file does not match the expected number."

    num_bins = 5
    _, ax = plt.subplots(figsize=(12, 6))

    colors = plt.cm.copper(np.linspace(0, 1, num_bins))

    indices = np.arange(num_criteria) * 1.5
    bar_width = 0.2

    bin_edges = np.linspace(0, 1, num_bins + 1)

    for i in range(num_bins):
        if i < num_bins - 1:
            bin_range = (bin_edges[i], bin_edges[i + 1])
            bin_counts = [
                np.mean((bin_range[0] <= scores[:, j]) & (scores[:, j] < bin_range[1]))
                for j in range(num_criteria)
            ]
        else:
            bin_range = (bin_edges[i], bin_edges[i + 1])
            bin_counts = [
                np.mean((bin_range[0] <= scores[:, j]) & (scores[:, j] <= bin_range[1]))
                for j in range(num_criteria)
            ]

        ax.bar(indices + i * bar_width, bin_counts, bar_width, label=f'Level {i}', color=colors[i])


    ax.set_ylabel('Proportion of Images', fontsize=13, fontweight='bold')
    #ax.set_title('Distribution of Distortion Scores for Images Processed Through the Distortion Pipeline', fontsize=15)
    ax.set_xticklabels([]) 
    ax.tick_params(axis='x', length=0) 

    for idx, criterion in enumerate(distortion_criteria):
        ax.annotate(criterion, (indices[idx] + num_bins * bar_width / 2, 0), xytext=(0, -11),
                    textcoords="offset points", ha='center', va='top', fontsize=11, arrowprops=dict(arrowstyle='-[, widthB=3.5, lengthB=0.3', lw=1.5, color='black', shrinkB=5), fontweight='bold')

    ax.legend(title="Score Range", loc='upper right', fontsize=13, title_fontsize=13)

    plt.show()

plot_grouped_distributions_from_npy('scores.npy')

In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np

def plot_grouped_distributions(output_file):
    with open(output_file, 'r') as file:
        scores = json.load(file)

    # Define the new order for distortion criteria
    distortion_criteria = ["background", "lighting", "focus", "orientation", "color_calibration", "resolution", "field_of_view"]
    criteria_scores = {criterion: [] for criterion in distortion_criteria}

    for image_scores in scores.values():
        for criterion, score in image_scores.items():
            criteria_scores[criterion].append(score)

    num_bins = 5
    _, ax = plt.subplots(figsize=(12, 6))

    n_groups = len(distortion_criteria)
    indices = np.arange(n_groups) * 1.5
    bar_width = 0.2

    colors = plt.cm.copper(np.linspace(0, 1, num_bins))

    bin_edges = np.linspace(0, 1, num_bins + 1)

    for i in range(num_bins):
        if i < num_bins - 1:
            bin_range = (bin_edges[i], bin_edges[i + 1])
            bin_means = {
                criterion: np.mean([(bin_range[0] <= score < bin_range[1]) for score in scores])
                for criterion, scores in criteria_scores.items()
            }
        else:
            bin_range = (bin_edges[i], bin_edges[i + 1])
            bin_means = {
                criterion: np.mean([(bin_range[0] <= score <= bin_range[1]) for score in scores])
                for criterion, scores in criteria_scores.items()
            }

        rects = ax.bar(indices + i * bar_width, list(bin_means.values()), bar_width,
                       label=f'Level {i}', color=colors[i])

    ax.set_ylabel('Proportion of Images', fontsize=13, fontweight='bold')
    #ax.set_title('Distribution of Distortion Scores for Manually Labeled Test Images', fontsize=15)
    ax.set_xticklabels([]) 
    ax.tick_params(axis='x', length=0) 
    distortion_criteria = ['Background', 'Lighting', 'Focus', 'Orientation', 'Color calibration', 'Resolution', 'Field of view']

    for idx, criterion in enumerate(distortion_criteria):
        ax.annotate(criterion, (indices[idx] + num_bins * bar_width / 2, 0), xytext=(0, -11),
                    textcoords="offset points", ha='center', va='top', fontsize=11, arrowprops=dict(arrowstyle='-[, widthB=3.5, lengthB=0.3', lw=1.5, color='black', shrinkB=5), fontweight='bold')

    #ax.legend(title="Score Range", loc='upper center', bbox_to_anchor=(0.5, 1.15), fontsize=13, title_fontsize=13)
    ax.legend(title="Score Range", loc='upper center', bbox_to_anchor=(0.25, 1.0), fontsize=13, title_fontsize=13)


    plt.show()

plot_grouped_distributions('scores.json')